## Notebook nettoyé de notre procédure de segmentation.

#### Mise en Place
On commence par importer les bibliothèques nécessaires. On s'appuyera sur torchio, une blibli dvlp pour l'imagerie médicale et nours permettant de visualiser des fichiers `.nii`

In [6]:
import torch
import torchio as tio

import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import filters, segmentation, morphology, measure, transform

from skimage import feature
import cv2
import scipy.signal as signal

from ipywidgets import interact
import ipywidgets as widgets

In [35]:
def load_patient(num):
    with open(f'database/training/patient{num:03}/info.cfg') as f:
        info = [line.replace("\n", "").replace(":", "").split(" ") for line in f]
        for i in range(len(info)):
            if info[i][1].isnumeric():
                info[i][1] = int(info[i][1])

    patient = tio.Subject(
        img=tio.ScalarImage(f'database/training/patient{num:03}/patient{num:03}_4d.nii.gz'),
        gt=tio.LabelMap(f'database/training/patient{num:03}/patient{num:03}_frame{dict(info)["ED"]:02}_gt.nii.gz'),
        info = dict(info)
    )
    return patient

In [ ]:
def tf_mask2(image4d, Tps, Slice, heartrate=2):
    fft_img = torch.fft.fft(image4d, axis=0)
    filtred = fft_img
    # interpolation frequance cardiaque
    heartrate_i = int(heartrate)
    heartrate_f = heartrate - heartrate_i
    filtred[:heartrate_i, :, :, :] = 0.0
    filtred[heartrate_i+1:, :, :, :] = 0.0
    filtred[heartrate_i] *= (1-heartrate_f)
    filtred[heartrate_i+1] *= heartrate_f

    timg2 = torch.abs(torch.fft.ifft(filtred, axis=0))

    mask = np.array(timg2[Tps, :, :, Slice])
    mask = np.uint8(2*mask/np.max(mask)*255)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(75,75))

    #mask = cv2.medianBlur(mask,3)
    mask = cv2.dilate(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(75,75)), 1)
    mask = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(95,95)), 1, borderType=cv2.BORDER_CONSTANT, borderValue=0)
    mask = cv2.blur(mask,(21, 21))

    img_m = np.array(image4d[Tps, :, :, Slice], dtype=np.float64)
    img_m = np.uint8(img_m/np.max(img_m)*np.float64(mask))
    img_m = cv2.medianBlur(img_m,5)
    return img_m, mask

In [37]:
Patient_étudié = 9
patient = load_patient(Patient_étudié)
t_img = patient.img.data.clone().detach()
t_gt = patient.gt.data.clone().detach()
img_m, mask = tf_mask2(t_img, heartrate=t_img.shape[0] / (patient.info["ES"] - patient.info["ED"]))


Indice_temps, Indice_coupe = patient.info["ED"], t_img.shape[3]//2
fig = plt.figure(figsize=(15,5))
fig.ticks = False
plt.subplot(131)
plt.imshow(t_img[Indice_temps, :, :, Indice_coupe], cmap='gray')
plt.title("Image d'origine")
plt.subplot(132)
plt.imshow(mask, cmap='gray')
plt.title("Masque")
plt.subplot(133)
plt.imshow(img_m, cmap='gray')
plt.title("Image masquée")
plt.tight_layout()
print("hr: ", t_img.shape[0] / (patient.info["ES"] - patient.info["ED"]))

TypeError: max() received an invalid combination of arguments - got (axis=NoneType, out=NoneType, ), but expected one of:
 * ()
 * (Tensor other)
 * (int dim, bool keepdim = False)
      didn't match because some of the keywords were incorrect: axis, out
 * (name dim, bool keepdim = False)
      didn't match because some of the keywords were incorrect: axis, out
